## Example: Overview over the main functionalities

This is an example on how to use the code to:

1. **Generate C code for the Jacobians and RK4 function**:
   - The first step is to generate C code for the Jacobians and the Runge-Kutta 4 (RK4) function.
   - After that, compile the generated C code into a dynamically linked library (`.so` file on Linux, `.dll` on Windows).


2. **Generate C code for the Controller**:
   - Once the Jacobians and RK4 functions are compiled, generate the C code for the controller.
   

3. **Run a Closed-Loop Simulation**:
   - Use the compiled controller in a closed-loop simulation to track a reference trajectory.
   - This step involves testing the controller in a simulated environment to see how well it performs in terms of trajectory tracking.


In [ ]:
from utils.CasadiJacobians import CasadiJacobians
from params import *

CreateJacobians = CasadiJacobians('params.json','Thorpy')


A_func,B_func,rk4_func = CreateJacobians.Thorpy_Gen()




### 1) **Generate C Code for the Jacobians and RK4 Function**
-Create folder `Codegen/Casadi`.
- Move the `A_func.c`, `B_func.c`, and `rk4_func.c` files into the `Codegen/Casadi` folder.

### 2) **Compile the C Code into a Dynamically Linked Library**

- Once the C code files are placed into the `Codegen/Casadi` folder, compile them using the following command:

    ```bash
    gcc -fPIC -shared rk4_func.c -o rk4_func.so
    ```

- Repeat the process for the other C files (`A_func.c` and `B_func.c`) in the same manner.
### Summary

- Once these functions are generated and compiled, they will be ready to use in the MPC controller and will not need to be regenerated unless there are changes to the rocket dynamics model.

The correct functioning of the compiled functions can be verified by running:

In [ ]:
CreateJacobians.Validate_Compiled_Functions(A_func, B_func, rk4_func)


### Running the Code Generation Script  

The following code snippet (which is the same as `main_Codegen_Thorpy`) can be executed to generate the MPC in C.  

After a successful code generation, the script will prompt whether to continue running. Since the primary purpose is to generate the MPC in C, it is recommended to type **"n"** to exit.  

Alternatively, typing **"y"** will continue running the Python version of the MPC. However, this is **not recommended**, as it is extremely slow.


In [ ]:
from mpc.MPC_Codegen_Thorpy import MPC_gen
from dynamics.RocketDynamics import RocketDynamics
import casadi as ca
import os

A_funcgen = ca.external("A_func", "Codegen/Casadi/A_func.so")
B_funcgen = ca.external("B_func", "Codegen/Casadi/B_func.so")
rk4_funcgen = ca.external("rk4_func", "Codegen/Casadi/rk4_func.so")


rocket_dyn = RocketDynamics('params.json')

ecos_dir = 'Codegen/ECOS'
if not os.path.exists(ecos_dir):
    os.makedirs(ecos_dir)
controller_dir = 'Codegen/ECOS/mpc_Thorpy_ECOS_Nmpc30_T3'

mpc = MPC_gen('params.json',rocket_dyn,A_funcgen,B_funcgen,rk4_funcgen,controller_dir)

x, u,error_vect = mpc.generate_C_code()

### Testing the Controller  

After successful code generation, the controller can be tested in a closed-loop simulation by running **`main_Thorpy`**.  


In [ ]:
from mpc.MPC_Tracker_Thorpy import MPC_Tracker
from dynamics.RocketDynamics import RocketDynamics
from simulation.ClosedLoop_Simulation import ClosedLoop_Simulation
from utils.plot_results import plot_results_tracker, plot_mpc_metrics
import casadi as ca

from Codegen.ECOS.mpc_Thorpy_ECOS_Nmpc30_T3.cpg_solver import cpg_solve

A_funcgen = ca.external("A_func", "Codegen/Casadi/A_func.so")
B_funcgen = ca.external("B_func", "Codegen/Casadi/B_func.so")
rk4_funcgen = ca.external("rk4_func", "Codegen/Casadi/rk4_func.so")


rocket_dyn = RocketDynamics('params.json')
controller_dir = 'Codegen/ECOS/mpc_Thorpy_ECOS_Nmpc30_T3/problem.pickle'


mpc = MPC_Tracker('params.json',rocket_dyn,A_funcgen,B_funcgen,rk4_funcgen,controller_dir,cpg_solve)
cl_sim = ClosedLoop_Simulation(mpc)

x, u,error_vect, solver_times, tracking_errors, max_tracking_error =cl_sim.simulate()



# save_folder = 'C:/Users/Utente/Desktop/Hopper_plots/Quaternions/T3_N30_Mass_Long2'

plot_results_tracker(mpc.t_ref, x, mpc.X_ref, u, mpc.U_ref)

plot_mpc_metrics(mpc.t_ref, solver_times, tracking_errors, max_tracking_error)




save_folder = None

plot_results_tracker(mpc.t_ref, x, mpc.X_ref, u, mpc.U_ref,save_folder)

plot_mpc_metrics(mpc.t_ref, solver_times, tracking_errors, max_tracking_error,save_folder)

